# Train your first Deep Reinforcement Learning Agent

PYTHON 3.9

In [2]:
import numpy as np

from collections import deque

import matplotlib.pyplot as plt
%matplotlib inline

# PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

# Hugging Face Hub
from huggingface_hub import notebook_login # To log to our Hugging Face account to be able to upload models to the Hub.
import imageio

El objetivo de este notebook es crear un sistema para entrenar los algoritmos que sea claro y escalable. Así que voy a tratar el menor número de celdas posibles

In [3]:
import sys #-- lo he necesitado para importar el paquete
sys.path.append('..')
import gym_hearthstone

## Argumentos
He añadido argumentos al environment para que sea más sencillo hacer el entrenamiento:

    action_type: Define el espacio de acción que se va a crear. Ahora mismo solo hay tres implementados.
    
        * "random"  -> Solo una acción possible (Tomar una acción aleatoria de las que son posibles).
        * "type_rd" -> 6 acciones posibles Realiza una acción aleatoria del tipo de acción devuelto por el DQN (en caso de tener alguna carta) y si no tengo ninguna carta del tipo de acción que ha devuelto el DQN entonces hace una acción aleatoria de entre las disponibles). NO TIENE SENTIDO PORQUE CONFUNDE AL DQN
        * "type"    -> 6 acciones posibles (Realiza una acción aleatoria del tipo de acción devuelto por el DQN (en caso de tener alguna carta) y si no tengo ninguna carta del tipo de acción que ha devuelto el DQN no hace nada. A continuación vuelve a invocar al DQN para obtener otro tipo de acción).
        
        
        
    reward_mode: Define el reward que devuelve el environment:
    
        * "simple"    -> Solo hay reward cuando se gana o se pierde.
        * "penalize"  -> Se penalizan las acciones que no son válidas.
        * "incentive" -> Se premian las acciones que son válidas.
        * "complex"   -> Se penalizan las acciones que no son válidas y se premian las válidas.


    steps: Por ahora usa el numero de steps para el nombre del fichero de salida. 
    
        

In [4]:
import gymnasium as gym
import gym_hearthstone

In [33]:
env_id = "Hearthstone-v1"
env = gym.make('Hearthstone-v1', action_type = "random", reward_mode = "simple")

[fireplace.entity]: Setting up game Game(players=(Player(name='Player1', hero=None), Player(name='Player2', hero=None)))
[fireplace.entity]: Tossing the coin... Player1 wins!
[fireplace.actions]: Player(name='Player1', hero=<Hero ('Garrosh Hellscream')>) triggering <TargetedAction: Summon(<Summon.CARD>=<HeroPower ('Armor Up!')>)> targeting [Player(name='Player1', hero=<Hero ('Garrosh Hellscream')>)]
[fireplace.actions]: Player1 summons [<HeroPower ('Armor Up!')>]
[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.actions]: Player(name='Player1', hero=<Hero ('Garrosh Hellscream')>) triggering <TargetedAction: Summon(<Summon.CARD>=<Hero ('Garrosh Hellscream')>)> targeting [Player(name='Player1', hero=<Hero ('Garrosh Hellscream')>)]
[fireplace.actions]: Player1 summons [<Hero ('Garrosh Hellscream')>]
[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.entity]: Player(name='Player1', hero=<Hero ('Garrosh Hellscream')>) shuffles

HearthstoneEnv - Version 0.2.0
Step.BEGIN_MULLIGAN
Step.MAIN_ACTION


In [34]:
# Get the state space and action space
s_size = len(list(env.observation_space))
print(s_size)
a_size = env.action_space.n
print(a_size)

364
1


In [35]:
state = env.reset()
state=np.array(list(state.values()))

[fireplace.entity]: Setting up game Game(players=(Player(name='Player1', hero=None), Player(name='Player2', hero=None)))
[fireplace.entity]: Tossing the coin... Player1 wins!
[fireplace.actions]: Player(name='Player1', hero=<Hero ('Rexxar')>) triggering <TargetedAction: Summon(<Summon.CARD>=<HeroPower ('Steady Shot')>)> targeting [Player(name='Player1', hero=<Hero ('Rexxar')>)]
[fireplace.actions]: Player1 summons [<HeroPower ('Steady Shot')>]
[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.actions]: Player(name='Player1', hero=<Hero ('Rexxar')>) triggering <TargetedAction: Summon(<Summon.CARD>=<Hero ('Rexxar')>)> targeting [Player(name='Player1', hero=<Hero ('Rexxar')>)]
[fireplace.actions]: Player1 summons [<Hero ('Rexxar')>]
[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.entity]: Player(name='Player1', hero=<Hero ('Rexxar')>) shuffles their deck
[fireplace.card]: <Minion ('Crackling Razormaw')> moves from <Zone.D

------------------------
>>> Episode 0
------------------------
Step.BEGIN_MULLIGAN
Step.MAIN_ACTION
>>> HANDP1:  7801 Crackling Razormaw
>>> HANDP1:  7799 Raptor Hatchling
>>> HANDP1:  2350 Houndmaster
>>> HANDP1:  2399 King Mukla
>>> HANDP2:  2416 Twilight Drake
>>> HANDP2:  2221 Mana Reservoir
>>> HANDP2:  7816 Primalfin Lookout
>>> HANDP2:  2823 The Coin
>>> HANDP2:  1367 Grimestreet Informant


In [36]:
# Resetear env y stats
import time
env.reset_stats()
env.reset()
start = time.time()

[fireplace.entity]: Setting up game Game(players=(Player(name='Player1', hero=None), Player(name='Player2', hero=None)))
[fireplace.entity]: Tossing the coin... Player2 wins!
[fireplace.actions]: Player(name='Player1', hero=<Hero ('Jaina Proudmoore')>) triggering <TargetedAction: Summon(<Summon.CARD>=<HeroPower ('Fireblast')>)> targeting [Player(name='Player1', hero=<Hero ('Jaina Proudmoore')>)]
[fireplace.actions]: Player1 summons [<HeroPower ('Fireblast')>]
[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.actions]: Player(name='Player1', hero=<Hero ('Jaina Proudmoore')>) triggering <TargetedAction: Summon(<Summon.CARD>=<Hero ('Jaina Proudmoore')>)> targeting [Player(name='Player1', hero=<Hero ('Jaina Proudmoore')>)]
[fireplace.actions]: Player1 summons [<Hero ('Jaina Proudmoore')>]
[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.entity]: Player(name='Player1', hero=<Hero ('Jaina Proudmoore')>) shuffles their deck
[f

------------------------
>>> Episode 0
------------------------
Step.BEGIN_MULLIGAN
Step.MAIN_ACTION
>>> HANDP1:  3033 WANTED!
>>> HANDP1:  2471 SI:7 Agent
>>> HANDP1:  7875 Bluegill Warrior
>>> HANDP1:  176 Evil Heckler
>>> HANDP2:  1859 Mirror Image
>>> HANDP2:  2817 Baron Rivendare
>>> HANDP2:  3070 Echo of Medivh
>>> HANDP2:  2823 The Coin
>>> HANDP2:  7894 Scarlet Crusader


In [37]:
class Policy(nn.Module):
    def __init__(self, s_size, a_size, h_size):
        super(Policy, self).__init__()
        self.fc1 = nn.Linear(s_size, h_size)
        self.fc2 = nn.Linear(h_size, h_size*2)
        self.fc3 = nn.Linear(h_size*2, h_size*2)
        self.fc4 = nn.Linear(h_size*2, a_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return F.softmax(x, dim=1)
    
    def act(self, state):
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        probs = self.forward(state).cpu()
        m = Categorical(probs)
        action = m.sample()
        return action.item(), m.log_prob(action)

In [38]:
def reinforce(policy, optimizer, n_training_episodes, max_t, gamma, print_every):
    # Help us to calculate the score during the training
    # max_t : longitud máxima del episodio

    steps_total=0
    scores_deque = deque(maxlen=100)  #double ended queue
    scores = []
    # Line 3 of pseudocode
    for i_episode in range(1, n_training_episodes+1):
        print("========================")
        print(">>> CURRENT EPISODE ",i_episode)
        print("========================")
        saved_log_probs = []
        rewards = []
        state = env.reset()
        state=np.array(list(state.values()))
        # Line 4 of pseudocode
        # En este bucle se realiza un episodio completo con un máximo de steps igual a max_t
        for t in range(max_t):
            steps_total=steps_total+1
            action, log_prob = policy.act(state)
            saved_log_probs.append(log_prob)
            state, reward, done, _ = env.step(action)
            state=np.array(list(state.values()))
            rewards.append(reward)
            if done:
                break 
        scores_deque.append(sum(rewards))
        scores.append(sum(rewards))
        
        # Line 6 of pseudocode: calculate the return
        returns = deque(maxlen=max_t) 
        n_steps = len(rewards) 
        # Compute the discounted returns at each timestep,
        # as 
        #      the sum of the gamma-discounted return at time t (G_t) + the reward at time t
        #
        # In O(N) time, where N is the number of time steps
        # (this definition of the discounted return G_t follows the definition of this quantity 
        # shown at page 44 of Sutton&Barto 2017 2nd draft)
        # G_t = r_(t+1) + r_(t+2) + ...
        
        # Given this formulation, the returns at each timestep t can be computed 
        # by re-using the computed future returns G_(t+1) to compute the current return G_t
        # G_t = r_(t+1) + gamma*G_(t+1)
        # G_(t-1) = r_t + gamma* G_t
        # (this follows a dynamic programming approach, with which we memorize solutions in order 
        # to avoid computing them multiple times)
        
        # This is correct since the above is equivalent to (see also page 46 of Sutton&Barto 2017 2nd draft)
        # G_(t-1) = r_t + gamma*r_(t+1) + gamma*gamma*r_(t+2) + ...
        
        
        ## Given the above, we calculate the returns at timestep t as: 
        #               gamma[t] * return[t] + reward[t]
        #
        ## We compute this starting from the last timestep to the first, in order
        ## to employ the formula presented above and avoid redundant computations that would be needed 
        ## if we were to do it from first to last.
        
        ## Hence, the queue "returns" will hold the returns in chronological order, from t=0 to t=n_steps
        ## thanks to the appendleft() function which allows to append to the position 0 in constant time O(1)
        ## a normal python list would instead require O(N) to do this.
        for t in range(n_steps)[::-1]:
            disc_return_t = (returns[0] if len(returns)>0 else 0)
            returns.appendleft( gamma*disc_return_t + rewards[t]   )    
            
        ## standardization of the returns is employed to make training more stable
        eps = np.finfo(np.float32).eps.item()
        ## eps is the smallest representable float, which is 
        # added to the standard deviation of the returns to avoid numerical instabilities        
        returns = torch.tensor(returns)
        returns = (returns - returns.mean()) / (returns.std() + eps)
        
        # Line 7:
        policy_loss = []
        for log_prob, disc_return in zip(saved_log_probs, returns):
            policy_loss.append(-log_prob * disc_return)
        policy_loss = torch.cat(policy_loss).sum()
        
        # Line 8: PyTorch prefers gradient descent 
        optimizer.zero_grad()
        policy_loss.backward()
        optimizer.step()
        
        if i_episode % print_every == 0:
            print('Episode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))
        
    return scores, steps_total

In [39]:
hearthstone_hyperparameters = {
    "h_size": 64,
    "n_training_episodes": 100,
    "n_evaluation_episodes": 5,
    "max_t": 400,
    "gamma": 0.99,
    "lr": 1e-4,
    "env_id": env_id,
    "state_space": s_size,
    "action_space": a_size,
}

In [40]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [41]:
# Create policy and place it to the device
hearthstone_policy = Policy(hearthstone_hyperparameters["state_space"], hearthstone_hyperparameters["action_space"], hearthstone_hyperparameters["h_size"]).to(device)
hearthstone_policy

Policy(
  (fc1): Linear(in_features=364, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=128, bias=True)
  (fc4): Linear(in_features=128, out_features=1, bias=True)
)

In [42]:
hearthstone_optimizer = optim.Adam(hearthstone_policy.parameters(), lr=hearthstone_hyperparameters["lr"])

In [ ]:
scores, steps_total = reinforce(hearthstone_policy,
                   hearthstone_optimizer,
                   hearthstone_hyperparameters["n_training_episodes"], 
                   hearthstone_hyperparameters["max_t"],
                   hearthstone_hyperparameters["gamma"], 
                   100)

[fireplace.entity]: Setting up game Game(players=(Player(name='Player1', hero=None), Player(name='Player2', hero=None)))
[fireplace.entity]: Tossing the coin... Player2 wins!
[fireplace.actions]: Player(name='Player1', hero=<Hero ('Anduin Wrynn')>) triggering <TargetedAction: Summon(<Summon.CARD>=<HeroPower ('Lesser Heal')>)> targeting [Player(name='Player1', hero=<Hero ('Anduin Wrynn')>)]
[fireplace.actions]: Player1 summons [<HeroPower ('Lesser Heal')>]
[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.actions]: Player(name='Player1', hero=<Hero ('Anduin Wrynn')>) triggering <TargetedAction: Summon(<Summon.CARD>=<Hero ('Anduin Wrynn')>)> targeting [Player(name='Player1', hero=<Hero ('Anduin Wrynn')>)]
[fireplace.actions]: Player1 summons [<Hero ('Anduin Wrynn')>]
[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.entity]: Player(name='Player1', hero=<Hero ('Anduin Wrynn')>) shuffles their deck
[fireplace.card]: <Spell (

>>> CURRENT EPISODE  1
------------------------
>>> Episode 0
------------------------
Step.BEGIN_MULLIGAN
Step.MAIN_ACTION
>>> HANDP1:  817 Weaponized Piñata
>>> HANDP1:  4516 Keeper of Uldaman
>>> HANDP1:  142 Maiden of the Lake
>>> HANDP1:  3676 Howling Commander
>>> HANDP2:  3637 Shadow Ascendant
>>> HANDP2:  3640 Obsidian Statue
>>> HANDP2:  7685 Sated Threshadon
>>> HANDP2:  2823 The Coin
>>> HANDP2:  1951 Stormpike Commando
------------------------------------------
>>> Current Step 1, Global 1, Average 1
------------------------------------------

>>> Before possible_actions RL agent 3:[[<Move.choice: 5>, 0], [<Move.choice: 5>, 1], [<Move.choice: 5>, 2]]
>>> Already Selected Actions RL agent 0:[]
>>> After possible_actions RL agent 3:[[<Move.choice: 5>, 0], [<Move.choice: 5>, 1], [<Move.choice: 5>, 2]]
>>> PLAYER: RL MODE: random. SELECTED ACTION [<Move.choice: 5>, 1]
>>> PLAYER: RL ACTION SELECTED
>>> Doing single action for RL Agent[<Move.choice: 5>, 1]
PLAYER :Player2 HAS CH

[fireplace.entity]: Player2 uses hero power <HeroPower ('Reinforce')> on None
[fireplace.entity]: Player2 pays 2 mana
[fireplace.actions]: <HeroPower ('Reinforce')> triggering <TargetedAction: Summon(<Summon.CARD>='CS2_101t')> targeting [Player(name='Player2', hero=<Hero ('Uther Lightbringer')>)]
[fireplace.actions]: Player2 summons [<Minion ('Silver Hand Recruit')>]
[fireplace.card]: <Minion ('Silver Hand Recruit')> moves from <Zone.SETASIDE: 6> to <Zone.PLAY: 1>
[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.entity]: Player2 ends turn 2
[fireplace.entity]: Player1 begins turn 3
[fireplace.entity]: Player1 is now at 1 mana crystals
[fireplace.actions]: Player(name='Player1', hero=<Hero ('Anduin Wrynn')>) triggering <TargetedAction: Draw()> targeting [Player(name='Player1', hero=<Hero ('Anduin Wrynn')>)]
[fireplace.entity]: Player1 draws <Minion ('Ice Rager')>
[fireplace.card]: <

------------------------------------------
>>> Current Step 2, Global 2, Average 2
------------------------------------------
>>> Moves Dictionary:  {'choice': [], 'play_card': [[<Move.play_card: 4>, 3, None, None], [<Move.play_card: 4>, 4, None, None]], 'heropower': [[<Move.hero_power: 1>, None, None]], 'minion_attack': [], 'hero_attack': [], 'end_turn': [[<Move.end_turn: 0>]]}

>>> Before possible_actions RL agent 4:[[<Move.play_card: 4>, 3, None, None], [<Move.play_card: 4>, 4, None, None], [<Move.hero_power: 1>, None, None], [<Move.end_turn: 0>]]
>>> Already Selected Actions RL agent 0:[]
>>> After possible_actions RL agent 4:[[<Move.play_card: 4>, 3, None, None], [<Move.play_card: 4>, 4, None, None], [<Move.hero_power: 1>, None, None], [<Move.end_turn: 0>]]
>>> PLAYER: RL MODE: random. SELECTED ACTION [<Move.hero_power: 1>, None, None]
>>> PLAYER: RL ACTION SELECTED
>>> Doing single action for RL Agent[<Move.hero_power: 1>, None, None]
>>> HANDP1:  817 Weaponized Piñata
>>> HANDP1

[fireplace.actions]: Player2 plays <Spell ('The Last Kaleidosaur')> (target=None, index=None)
[fireplace.entity]: Player2 pays 1 mana
[fireplace.card]: <Spell ('The Last Kaleidosaur')> moves from <Zone.HAND: 3> to <Zone.PLAY: 1>
[fireplace.actions]: <Spell ('The Last Kaleidosaur')> triggering <TargetedAction: Battlecry(<Battlecry.TARGET>=None)> targeting [<Spell ('The Last Kaleidosaur')>]
[fireplace.actions]: Activating <Spell ('The Last Kaleidosaur')> action targeting None
[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.actions]: Player2 plays <Minion ('Cloakscale Chemist')> (target=None, index=None)
[fireplace.entity]: Player2 pays 2 mana
[fireplace.card]: <Minion ('Cloakscale Chemist')> moves from <Zone.HAND: 3> to <Zone.PLAY: 1>
[fireplace.actions]: <Minion ('Cloakscale Chemist')> triggering <TargetedAction: Battlecry(<Battlecry.TARGET>=None)> targeting [<Minion ('Cloakscale C

...

[fireplace.actions]: Player2 plays <Minion ('Knife Juggler')> (target=None, index=None)
[fireplace.entity]: Player2 pays 2 mana
[fireplace.card]: <Minion ('Knife Juggler')> moves from <Zone.HAND: 3> to <Zone.PLAY: 1>
[fireplace.actions]: <Minion ('Knife Juggler')> triggering <TargetedAction: Battlecry(<Battlecry.TARGET>=None)> targeting [<Minion ('Knife Juggler')>]
[fireplace.actions]: Activating <Minion ('Knife Juggler')> action targeting None
[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.entity]: Empty stack, refreshing auras and processing deaths


 5449 Knife Juggler
>>> HANDP1:  3003 Witchwood Grizzly
>>> HANDP1:  2651 Nat Pagle
>>> HANDP1:  7703 Tortollan Shellraiser
>>> HANDP1:  801 Zerek, Master Cloner
>>> FIELDP1:  24 Spawn of Shadows
>>> FIELDP1:  5687 Nat, the Darkfisher
>>> HANDP2:  3002 Lifedrinker
>>> HANDP2:  3179 Force-Tank MAX
>>> HANDP2:  3035 Genn Greymane
>>> HANDP2:  1467 Naga Corsair
>>> HANDP2:  1864 Water Elemental
>>> HANDP2:  1220 Astromancer Solarian
>>> FIELDP2:  3174 Ship's Cannon
------------------------------------------
>>> Current Step 14, Global 85, Average 42
------------------------------------------
>>> Moves Dictionary:  {'choice': [], 'play_card': [[<Move.play_card: 4>, 0, None, None], [<Move.play_card: 4>, 2, None, None], [<Move.play_card: 4>, 4, None, None], [<Move.play_card: 4>, 5, None, None]], 'heropower': [], 'minion_attack': [[<Move.minion_attack: 2>, 0, 0], [<Move.minion_attack: 2>, 0, 1]], 'hero_attack': [], 'end_turn': [[<Move.end_turn: 0>]]}

>>> Before possible_actions RL agent 7:[[

[fireplace.actions]: <Minion ('Spawn of Shadows')> attacks <Hero ('Jaina Proudmoore')>
[fireplace.actions]: <Minion ('Spawn of Shadows')> triggering <TargetedAction: Hit(<Hit.AMOUNT>=5)> targeting [<Hero ('Jaina Proudmoore')>]
[fireplace.actions]: <Minion ('Spawn of Shadows')> triggering <TargetedAction: Predamage(<Predamage.AMOUNT>=5)> targeting [<Hero ('Jaina Proudmoore')>]
[fireplace.actions]: <Minion ('Spawn of Shadows')> triggering <TargetedAction: Damage()> targeting [<Hero ('Jaina Proudmoore')>]
[fireplace.entity]: Empty stack, refreshing auras and processing deaths


 3003 Witchwood Grizzly
>>> HANDP1:  2651 Nat Pagle
>>> HANDP1:  7703 Tortollan Shellraiser
>>> HANDP1:  801 Zerek, Master Cloner
>>> FIELDP1:  24 Spawn of Shadows
>>> FIELDP1:  5687 Nat, the Darkfisher
>>> FIELDP1:  5449 Knife Juggler
>>> HANDP2:  3002 Lifedrinker
>>> HANDP2:  3179 Force-Tank MAX
>>> HANDP2:  3035 Genn Greymane
>>> HANDP2:  1467 Naga Corsair
>>> HANDP2:  1864 Water Elemental
>>> HANDP2:  1220 Astromancer Solarian
>>> FIELDP2:  3174 Ship's Cannon
------------------------------------------
>>> Current Step 15, Global 86, Average 43
------------------------------------------
>>> Moves Dictionary:  {'choice': [], 'play_card': [[<Move.play_card: 4>, 0, None, None], [<Move.play_card: 4>, 3, None, None]], 'heropower': [], 'minion_attack': [[<Move.minion_attack: 2>, 0, 0], [<Move.minion_attack: 2>, 0, 1]], 'hero_attack': [], 'end_turn': [[<Move.end_turn: 0>]]}

>>> Before possible_actions RL agent 5:[[<Move.play_card: 4>, 0, None, None], [<Move.play_card: 4>, 3, None, None], 

[fireplace.entity]: Player2 ends turn 10
[fireplace.entity]: Player1 begins turn 11
[fireplace.actions]: <Minion ('Nat, the Darkfisher')> triggers off <Action: BeginTurn(<BeginTurn.PLAYER>=Player(name='Player1', hero=<Hero ('Jaina Proudmoore')>))> from Game(players=(Player(name='Player1', hero=<Hero ('Jaina Proudmoore')>), Player(name='Player2', hero=<Hero ('Anduin Wrynn')>)))
[fireplace.actions]: <Minion ('Nat, the Darkfisher')> triggering <TargetedAction: Draw()> targeting [Player(name='Player1', hero=<Hero ('Jaina Proudmoore')>)]
[fireplace.entity]: Player1 draws <Minion ('Emerald Hive Queen')>
[fireplace.card]: <Minion ('Emerald Hive Queen')> moves from <Zone.DECK: 2> to <Zone.HAND: 3>
[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.entity]: Player1 is now at 5 mana crystals
[fireplace.actions]: Player(name='Player1', hero=<Hero ('Jaina Proudmoore')>) triggering <TargetedAction: Draw()> targeting [Player(name='Player1', hero=<Hero ('Jaina Proudmoo

In [ ]:
## Calcular estadísticas
env.display_stats()

end=time.time()
total_time = end-start

if(env.curr_episode > 0):
    avg_time = (end-start) / env.curr_episode
    avg_steps = steps_total / env.curr_episode
else:
    avg_time = (end-start) 
    avg_steps = steps_total 


if (env.wins + env.losses > 0):
    win_rate = env.wins / (env.wins + env.losses)
else:
    win_rate = 0

print("Avg steps per game: " + str(avg_steps))
print("Total running time: {:.2f} and running time per game: {:.2f}".format(total_time, avg_time) )

env.close()
